In [2]:
import os
import sys
import traci
import pandas as pd
import numpy as np
from itertools import product
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from xml.dom import minidom

In [3]:
csv_file = 'synthetic_tripinfo_data.csv'
data =pd.read_csv(csv_file)
data =pd.DataFrame(data)

In [6]:
print(data.head(5))

In [7]:
start_range=5401
end_range  =7200
school_edges = ["27", "-27", "28", "-28", "29", "-29", "30", "-30", "31", "-31"] 
data['depart'] = pd.to_numeric(data['depart'])
filtered_df    = data[(data['depart'] >= start_range) & (data['depart'] <= end_range)]
filtered_df = filtered_df[filtered_df['arrivalLane'].isin(school_edges)]
other_df = data[(data['depart'] < start_range) | (data['depart'] > end_range)]
other_df = other_df[~other_df['arrivalLane'].isin(school_edges)]
print(data.shape[0])
print(filtered_df.shape[0])
print(other_df.shape[0])

print(filtered_df)
print(other_df)

In [8]:
import numpy as np
iterations = 44  
np.random.seed(42)
updated_dfs = []
updated_dfs.append(filtered_df)
for i in range(iterations):
    rows_to_remove = filtered_df.sample(n=2).index
    updated_df = filtered_df.drop(rows_to_remove)
    updated_dfs.append(updated_df)
    filtered_df = updated_df.copy()
    
for i, df in enumerate(updated_dfs):
    print(f"Iteration {i+1}:\n{df}\n")

In [9]:
combined_dfs = [pd.concat([other_df, df], ignore_index=True) for df in updated_dfs]
combined_dfs = [df.sort_values(by='depart') for df in combined_dfs]

In [10]:
def df_to_xml2(df):
    root = ET.Element("routes")
    root.set("xmlns:xsi", "http://www.w3.org/2001/XMLSchema-instance")
    root.set("xsi:noNamespaceSchemaLocation", "http://sumo.dlr.de/xsd/routes_file.xsd")
    vtype_1 = ET.SubElement(root, "vType", id="type1", length="5", minGap="2.5", maxSpeed="70",
                            vClass="passenger", accel="2.6", decel="4.5", emergencyDecel="9", carFollowModel="IDM",
                            delta="5", stepping="1", tau="1", speedDev="0.1", emissionClass="PHEMlight/PC_G_EU4")

    for index, row in df.iterrows():
        flow = ET.SubElement(root, "trip")
        flow.set("id", str(row['id']))
        flow.set("depart", str(row['depart']))
        flow.set("from", str(row['from']))
        flow.set("to", str(row['to']))
        flow.set("departLane", str(row['departLane']))
        flow.set("arrivalLane", str(row['arrivalLane']))
        flow.set("type", str(row['type']))

    tree = ET.ElementTree(root)
    xml_str = minidom.parseString(ET.tostring(root)).toprettyxml(indent="  ")

    return xml_str

In [11]:
xml_list = [df_to_xml2(df) for df in combined_dfs]

In [12]:
print(xml_list[0])

In [13]:
if xml_list:
    first_xml_content = xml_list[0]

    # Specify the file path where you want to save the XML
    file_path = ".\IDM_first_demand.xml"

    with open(file_path, "w") as xml_file:
        xml_file.write(first_xml_content)

    print(f"First XML element has been saved to '{file_path}'")
else:
    print("xml_list is empty.")

In [15]:
tree = ET.parse(".\IDM_first_demand.xml")
root = tree.getroot()
trip_elements = root.findall('.//trip')
number_of_trips = len(trip_elements)
print(f"Number of trip IDs: {number_of_trips}")

In [16]:
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please set the environment variable 'SUMO_HOME'")
sumo_binary = "C:/Program Files (x86)/Eclipse/Sumo/bin/sumo"
sumocfg_file = ".\configuration.sumocfg"

In [17]:
import hashlib
serial_numbers = list(range(0,44))
for index, demand in enumerate(xml_list):
    #print(demand)
    input_file_path = "IDM_first_demand.xml"
        with open(input_file_path, "w") as xml_file:
        xml_file.write(demand)
    hash_object = hashlib.sha1(demand.encode())
    hash_value = hash_object.hexdigest()
    tree = ET.parse("additional.xml")
    root = tree.getroot()

    edge_data_element = root.find('.//edgeData[@id="1"]')

    if edge_data_element is not None:
        edge_data_element.set('file', f"edge_emission_{hash_value}_serial_{serial_numbers[index]}.xml")
        tree.write("additional.xml")
    else:
        print("edgeData element with id='1' not found in the XML.")

    output_file_path = f"tripinfos_{hash_value}_serial_{serial_numbers[index]}.xml"
    output_file_path2 = f"edgedata_{hash_value}_serial_{serial_numbers[index]}.xml"
    output_file_path3 = f"fcd_{hash_value}_serial_{serial_numbers[index]}.xml"
    output_file_path4 = f"emission_{hash_value}_serial_{serial_numbers[index]}.xml"
    traci.start([sumo_binary, "-c", sumocfg_file, "--tripinfo-output", output_file_path,"--edgedata-output",output_file_path2,"--fcd-output",output_file_path3,"--emission-output",output_file_path4])

    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()

    traci.close()
    sys.stdout.flush()
    xmldoc = ET.parse(output_file_path)

In [ ]:
traci.close()